In [232]:
import pandas as pd
from datetime import date

In [233]:
today = date.today()
today = pd.to_datetime(today)
print(today)

2022-03-10 00:00:00


In [234]:
!ls

calculate.ipynb
code # 2.png
salesorders.csv
select stockid, trandate from stockmoves.sql
stockmoves.csv


In [235]:
stockmoves = pd.read_csv('stockmoves.csv')

salesorder = pd.read_csv('salesorders.csv')

In [236]:
stockmoves.trandate = pd.to_datetime(stockmoves.trandate)
stockmoves.stockid = stockmoves.stockid.astype('string')
stockmoves

,stockid,trandate,total_qty
0,1,2020-02-18,-60
1,1,2022-03-09,1000
2,1234,2022-03-09,19
3,BBF,2020-02-17,10
4,BBF,2020-02-18,80
5,BOMBEE,2020-02-18,10
6,cfr00213,2020-02-18,10
7,CR000002,2020-02-17,20
8,CR000002,2020-02-18,10
9,THIS IS A TEST,2022-03-09,20000000


In [237]:
salesorder.deliverydate = pd.to_datetime(salesorder.deliverydate)
salesorder

,stockid,deliverydate,qty
0,CR000002,2020-03-17,1
1,cfr00213,2020-03-18,1
2,BBF,2020-02-18,20
3,BBF,2020-02-18,30
4,This is a test,2022-04-09,1
5,BBF,2022-03-09,12
6,This is a test,2022-03-09,13
7,This is a test,2022-03-10,14


In [238]:
stock_move_in = stockmoves[stockmoves.total_qty>0]
stock_move_out = stockmoves[stockmoves.total_qty<0]

In [239]:
stock_move_in

,stockid,trandate,total_qty
1,1,2022-03-09,1000
2,1234,2022-03-09,19
3,BBF,2020-02-17,10
4,BBF,2020-02-18,80
5,BOMBEE,2020-02-18,10
6,cfr00213,2020-02-18,10
7,CR000002,2020-02-17,20
8,CR000002,2020-02-18,10
9,THIS IS A TEST,2022-03-09,20000000


In [240]:
stock_move_out

,stockid,trandate,total_qty
0,1,2020-02-18,-60
10,THIS IS A TEST,2022-03-10,-1000


In [241]:
for i in stock_move_in.values.tolist():
    print(i)

['1', Timestamp('2022-03-09 00:00:00'), 1000]
['1234', Timestamp('2022-03-09 00:00:00'), 19]
['BBF', Timestamp('2020-02-17 00:00:00'), 10]
['BBF', Timestamp('2020-02-18 00:00:00'), 80]
['BOMBEE', Timestamp('2020-02-18 00:00:00'), 10]
['cfr00213', Timestamp('2020-02-18 00:00:00'), 10]
['CR000002', Timestamp('2020-02-17 00:00:00'), 20]
['CR000002', Timestamp('2020-02-18 00:00:00'), 10]
['THIS IS A TEST', Timestamp('2022-03-09 00:00:00'), 20000000]


In [242]:
stock = []

for i in stock_move_in.values.tolist():
    '''
        ['1' '3/9/2022' 1000]
        ['1234' '3/9/2022' 19]
        ['BBF' '2/17/2020' 10]
        ['BBF' '2/18/2020' 80]
        ['BOMBEE' '2/18/2020' 10]
        ['cfr00213' '2/18/2020' 10]
        ['CR000002' '2/17/2020' 20]
        ['CR000002' '2/18/2020' 10]
        ['THIS IS A TEST' '3/9/2022' 20000000]
    '''
    
    # if [0] in 
    for j in stock_move_out.values.tolist():
        if i[0] in j[0]:
            i.append(j[1]) # add timestamp move out
            i.append(j[2]) # add qty
        # if i[0] not in j[0]:
        #     print('hello')
        #     i.append(today)
        #     i.append(0)
        # elif i[0] not in j[0]:
        #     i.append(today)
        #     i.append(0)
    # print(i)
    stock.append(i)
print(stock)


[['1', Timestamp('2022-03-09 00:00:00'), 1000, Timestamp('2020-02-18 00:00:00'), -60], ['1234', Timestamp('2022-03-09 00:00:00'), 19], ['BBF', Timestamp('2020-02-17 00:00:00'), 10], ['BBF', Timestamp('2020-02-18 00:00:00'), 80], ['BOMBEE', Timestamp('2020-02-18 00:00:00'), 10], ['cfr00213', Timestamp('2020-02-18 00:00:00'), 10], ['CR000002', Timestamp('2020-02-17 00:00:00'), 20], ['CR000002', Timestamp('2020-02-18 00:00:00'), 10], ['THIS IS A TEST', Timestamp('2022-03-09 00:00:00'), 20000000, Timestamp('2022-03-10 00:00:00'), -1000]]


In [275]:

stock_move_in_and_out = pd.DataFrame(stock,columns=[['stockid','trandate_in','move_in_qty','trandate_out','move_out_qty']])

In [276]:
stock_move_in_and_out.trandate_out = stock_move_in_and_out.trandate_out.fillna(today) # filled by current datetime
stock_move_in_and_out.move_out_qty = stock_move_in_and_out.move_out_qty.fillna(0) # amount

In [277]:
stock_move_in_and_out['move_out_qty'] = stock_move_in_and_out['move_out_qty'].astype('int64')


In [278]:
stock_move_in_and_out.dtypes

stockid                 object
trandate_in     datetime64[ns]
move_in_qty              int64
trandate_out    datetime64[ns]
move_out_qty             int64
dtype: object

In [279]:
stock_move_in_and_out['item_life'] = stock_move_in_and_out['trandate_out'].values - stock_move_in_and_out['trandate_in'].values
stock_move_in_and_out['remain_qty'] = stock_move_in_and_out['move_in_qty'].values + stock_move_in_and_out['move_out_qty'].values
stock_move_in_and_out

,stockid,trandate_in,move_in_qty,trandate_out,move_out_qty,item_life,remain_qty
0,1,2022-03-09,1000,2020-02-18,-60,-750 days,940
1,1234,2022-03-09,19,2022-03-10,0,1 days,19
2,BBF,2020-02-17,10,2022-03-10,0,752 days,10
3,BBF,2020-02-18,80,2022-03-10,0,751 days,80
4,BOMBEE,2020-02-18,10,2022-03-10,0,751 days,10
5,cfr00213,2020-02-18,10,2022-03-10,0,751 days,10
6,CR000002,2020-02-17,20,2022-03-10,0,752 days,20
7,CR000002,2020-02-18,10,2022-03-10,0,751 days,10
8,THIS IS A TEST,2022-03-09,20000000,2022-03-10,-1000,1 days,19999000


In [280]:
stock_move_in_and_out['item_life'] = stock_move_in_and_out['item_life'].astype('timedelta64[D]')
stock_move_in_and_out['item_life'] = stock_move_in_and_out['item_life'].astype('int64')

In [281]:
stock_move_in_and_out.dtypes


stockid                 object
trandate_in     datetime64[ns]
move_in_qty              int64
trandate_out    datetime64[ns]
move_out_qty             int64
item_life                int64
remain_qty               int64
dtype: object

In [282]:
item = []
for i in stock_move_in_and_out.item_life.values:
    if i < 0:
        item.append(0)
    else:
        item.append(i[0])

In [283]:
item

[0, 1, 752, 751, 751, 751, 752, 751, 1]

In [284]:
stock_move_in_and_out['item_life'] = item

In [285]:
stock_move_in_and_out.stockid = stock_move_in_and_out.stockid.astype('string')
stock_move_in_and_out

,stockid,trandate_in,move_in_qty,trandate_out,move_out_qty,item_life,remain_qty
0,1,2022-03-09,1000,2020-02-18,-60,0,940
1,1234,2022-03-09,19,2022-03-10,0,1,19
2,BBF,2020-02-17,10,2022-03-10,0,752,10
3,BBF,2020-02-18,80,2022-03-10,0,751,80
4,BOMBEE,2020-02-18,10,2022-03-10,0,751,10
5,cfr00213,2020-02-18,10,2022-03-10,0,751,10
6,CR000002,2020-02-17,20,2022-03-10,0,752,20
7,CR000002,2020-02-18,10,2022-03-10,0,751,10
8,THIS IS A TEST,2022-03-09,20000000,2022-03-10,-1000,1,19999000


In [286]:
for i in stock_move_in_and_out.stockid.values.tolist():
    print(type(i[0]))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [290]:
stock_move_in_and_out.columns = ['stockid','trandate_in','move_in_qty','trandate_out','move_out_qty','item_life','remain_qty']

In [295]:
stock_move_in_and_out

,stockid,trandate_in,move_in_qty,trandate_out,move_out_qty,item_life,remain_qty
0,1,2022-03-09,1000,2020-02-18,-60,0,940
1,1234,2022-03-09,19,2022-03-10,0,1,19
2,BBF,2020-02-17,10,2022-03-10,0,752,10
3,BBF,2020-02-18,80,2022-03-10,0,751,80
4,BOMBEE,2020-02-18,10,2022-03-10,0,751,10
5,cfr00213,2020-02-18,10,2022-03-10,0,751,10
6,CR000002,2020-02-17,20,2022-03-10,0,752,20
7,CR000002,2020-02-18,10,2022-03-10,0,751,10
8,THIS IS A TEST,2022-03-09,20000000,2022-03-10,-1000,1,19999000


In [301]:
stock_with_life_time = stock_move_in_and_out.groupby('stockid').agg({'trandate_in':'max','item_life':'mean','remain_qty':'sum'}).reset_index()

In [314]:

stock_with_life_time.stockid = [i.upper() for i in stock_with_life_time.stockid.values.tolist()] # chagne to uppercase
salesorder.stockid = [i.upper() for i in salesorder.stockid.values.tolist()]

In [320]:
stock_with_life_time

,stockid,trandate_in,item_life,remain_qty
0,1,2022-03-09,0.0,940
1,1234,2022-03-09,1.0,19
2,BBF,2020-02-18,751.5,90
3,BOMBEE,2020-02-18,751.0,10
4,CR000002,2020-02-18,751.5,30
5,THIS IS A TEST,2022-03-09,1.0,19999000
6,CFR00213,2020-02-18,751.0,10


In [316]:
salesorder

,stockid,deliverydate,qty
0,CR000002,2020-03-17,1
1,CFR00213,2020-03-18,1
2,BBF,2020-02-18,20
3,BBF,2020-02-18,30
4,THIS IS A TEST,2022-04-09,1
5,BBF,2022-03-09,12
6,THIS IS A TEST,2022-03-09,13
7,THIS IS A TEST,2022-03-10,14


In [323]:
sale_with_stock = salesorder.merge(stock_with_life_time[['stockid','trandate_in','item_life']],left_on='stockid',right_on='stockid',how='left')

In [325]:
sale_with_stock['item_life_to_sale'] = sale_with_stock.deliverydate - sale_with_stock.trandate_in

In [329]:
sale_with_stock.item_life_to_sale = sale_with_stock.item_life_to_sale.astype('timedelta64[D]')
sale_with_stock.item_life_to_sale = sale_with_stock.item_life_to_sale.astype('int') # chagne to int

In [331]:
sale_with_stock

,stockid,deliverydate,qty,trandate_in,item_life,item_life_to_sale
0,CR000002,2020-03-17,1,2020-02-18,751.5,28
1,CFR00213,2020-03-18,1,2020-02-18,751.0,29
2,BBF,2020-02-18,20,2020-02-18,751.5,0
3,BBF,2020-02-18,30,2020-02-18,751.5,0
4,THIS IS A TEST,2022-04-09,1,2022-03-09,1.0,31
5,BBF,2022-03-09,12,2020-02-18,751.5,750
6,THIS IS A TEST,2022-03-09,13,2022-03-09,1.0,0
7,THIS IS A TEST,2022-03-10,14,2022-03-09,1.0,1


In [334]:
sale_with_stock_lifetime = sale_with_stock.groupby('stockid').agg({'item_life':'mean','item_life_to_sale':'mean'}).reset_index()

In [336]:
# sale_with_stock_lifetime['average_lifetime'] = (sale_with_stock_lifetime['item_life']+sale_with_stock_lifetime['item_life_to_sale'])/2

In [339]:
sale_with_stock_lifetime

,stockid,item_life,item_life_to_sale,average_lifetime
0,BBF,751.5,250.000000,500.750000
1,CFR00213,751.0,29.000000,390.000000
2,CR000002,751.5,28.000000,389.750000
3,THIS IS A TEST,1.0,10.666667,5.833333


In [341]:
stock_with_lifetime_and_sale_with_lifetime = stock_with_life_time.merge(right=sale_with_stock_lifetime[['stockid','item_life_to_sale']], left_on='stockid', right_on='stockid', how='left')

In [344]:
stock_with_lifetime_and_sale_with_lifetime = stock_with_lifetime_and_sale_with_lifetime.fillna(0)

In [346]:
stock_with_lifetime_and_sale_with_lifetime['average_lifetime'] = (stock_with_lifetime_and_sale_with_lifetime['item_life']+stock_with_lifetime_and_sale_with_lifetime['item_life_to_sale'])/2

In [347]:
stock_with_lifetime_and_sale_with_lifetime

,stockid,trandate_in,item_life,remain_qty,item_life_to_sale,average_lifetime
0,1,2022-03-09,0.0,940,0.000000,0.000000
1,1234,2022-03-09,1.0,19,0.000000,0.500000
2,BBF,2020-02-18,751.5,90,250.000000,500.750000
3,BOMBEE,2020-02-18,751.0,10,0.000000,375.500000
4,CR000002,2020-02-18,751.5,30,28.000000,389.750000
5,THIS IS A TEST,2022-03-09,1.0,19999000,10.666667,5.833333
6,CFR00213,2020-02-18,751.0,10,29.000000,390.000000
